<a href="https://colab.research.google.com/github/mitsuo/juntendo-hds/blob/main/ECG_pytorch_1d_conv_%E6%94%B9.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
import random
import time
import pandas as pd
import numpy as np
from scipy import signal
from scipy.signal import find_peaks, resample
import matplotlib.pyplot as plt
import os
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

print('Packages Loaded')

# 心電図データのダウンロード
今回は 全国医療AIコンテスト 2021 (https://www.kaggle.com/competitions/ai-medical-contest-2021/ ) の心電図データをダウンロードします。このデータは心電図から心筋梗塞かどうかを判定するタスクのためのものです。

まずデータをダウンロードしましょう。下のセルでは `ai-medical-contest-2021` というディレクトリをデフォルトのディレクトリ `/content` の下に作成し、そこに関連データをダウンロードしています。

In [ ]:
# ! を先頭につけると一時的に適応される。例えばワーキングディレクトリの移動をしてもその後のコマンドには適応されない。
# ai-medical-contest-2021 ディレクトリを作成します。
!rm -rf /content/ai-medical-contest-2021
!mkdir /content/ai-medical-contest-2021

# % を先頭につけると永続的に適応される。ワーキングディレクトリの移動をしてもその後も適応される。
# ai-medical-contest-2021 ディレクトリに移動します。
%cd /content/ai-medical-contest-2021
!pwd
!ls

# 心電図データのダウンロード
!wget http://mitsuo.nishizawa.com/juntendo/ai-medical-contest-2021.zip
!unzip ai-medical-contest-2021.zip
!ls

In [ ]:
data_dir = '/content/ai-medical-contest-2021'
train_path = f'{data_dir}/train.csv'
test_path = f'{data_dir}/test.csv'

col_target = 'target'
col_index = 'Id'
col_features = ['age', 'sex', 'label_type']

SEED = 42
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [ ]:
df_train = pd.read_csv(train_path)
print("df_train.shape", df_train.shape)
df_train.head()

In [ ]:
df_test = pd.read_csv(test_path)
print("df_test.shape", df_test.shape)
df_test.head()

In [ ]:
df_traintest = pd.concat([df_train, df_test]).reset_index(drop=True)
df_traintest['path'] = df_traintest['Id'].apply(lambda x: f"{data_dir}/ecg/{x}.npy")
print(df_traintest['path'][0])
df_traintest.head()

# 前処理

In [ ]:
df_traintest['sex'] = df_traintest['sex'].replace('female', 0)
df_traintest['sex'] = df_traintest['sex'].replace('male', 1)
df_traintest['sex'] = df_traintest['sex'].astype(int)

df_traintest['label_type'] = df_traintest['label_type'].replace('human', 0)
df_traintest['label_type'] = df_traintest['label_type'].replace('auto', 1)
df_traintest['label_type'] = df_traintest['label_type'].astype(int)
df_traintest.head()
df_traintest['sex'] = df_traintest['sex'].replace('female', 0)
df_traintest['sex'] = df_traintest['sex'].replace('male', 1)
df_traintest['sex'] = df_traintest['sex'].astype(int)

df_traintest['label_type'] = df_traintest['label_type'].replace('human', 0)
df_traintest['label_type'] = df_traintest['label_type'].replace('auto', 1)
df_traintest['label_type'] = df_traintest['label_type'].astype(int)

df_traintest['age'] = (df_traintest['age'] - df_traintest['age'].min()) / (df_traintest['age'].max() - df_traintest['age'].min())
df_traintest.head()

In [ ]:
df_train = df_traintest.iloc[:len(df_train)]
df_test = df_traintest.iloc[len(df_train):].reset_index(drop=True)
print(df_train.shape, df_test.shape)

In [ ]:
ecg_train = np.zeros([len(df_train), 800, 12], np.float32)
for i in range(len(df_train)):
    path_tmp = df_train['path'][i]
    ecg_tmp = np.load(path_tmp)
    ecg_train[i] = ecg_tmp

ecg_test = np.zeros([len(df_test), 800, 12], np.float32)
for i in range(len(df_test)):
    path_tmp = df_test['path'][i]
    ecg_tmp = np.load(path_tmp)
    ecg_test[i] = ecg_tmp

ecg_train = ecg_train.transpose(0, 2, 1)
ecg_test = ecg_test.transpose(0, 2, 1)
print("ecg_train.shape: {}".format(ecg_train.shape))
print("ecg_test.shape: {}".format(ecg_test.shape))

In [ ]:
target_train = df_train[col_target].values.astype(np.float32)
print("target_train.shape: {}".format(target_train.shape))

# DA

In [ ]:
def stretch(x, l):
    y = resample(x, l)
    if l < 800:
        y_ = np.zeros(shape=(800, ))
        y_[:l] = y
    else:
        y_ = y[:800]
    return y_

def amplify(x, alpha):
    factor = -alpha*x + (1+alpha)
    return x*factor

def stretch_twelve(ecg):
    l = int(800 * (1 + (random.random()-0.5)/3))
    *y_, = map(stretch, ecg, [l] * 12)
    return np.array(y_, dtype=np.float32)

def amplify_twelve(ecg):
    alpha = (random.random()-0.5)
    *y_, = map(amplify, ecg, [alpha] * 12)
    return np.array(y_, dtype=np.float32)

In [ ]:
def get_train_transforms():
    return transforms.Compose([
        transforms.Lambda(amplify_twelve),
        transforms.Lambda(stretch_twelve),
        transforms.ToTensor(),
    ])
def get_valid_transforms():
    return transforms.Compose([
        transforms.ToTensor(),
    ])

# Dataset

In [ ]:
class ECGDataset(Dataset):
    def __init__(self, X, X_add=None, y=None, train=True, transforms=None):
        super().__init__()
        self.X = X
        self.X_add = X_add
        self.y = y
        self.train = train
        self.transforms = transforms

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        X_trans = self.transforms(self.X[index])[0]
        if self.X_add is not None:
            X_add = torch.tensor(self.X_add[index], dtype=torch.float)
        else:
            X_add = None
        if self.train == True:
            return X_trans, X_add, torch.tensor(self.y[index], dtype=torch.float)
        else:
            return X_trans, X_add

# model

In [ ]:
class Anomaly_Classifier(nn.Module):
    def __init__(self, input_size,num_classes):
        super(Anomaly_Classifier, self).__init__()

        self.conv= nn.Conv1d(in_channels=input_size, out_channels=32, kernel_size=5,stride=1)

        self.conv_pad = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=5,stride=1,padding=2)
        self.drop_50 = nn.Dropout(p=0.5)

        self.maxpool = nn.MaxPool1d(kernel_size=5,stride=2)

        self.dense1 = nn.Linear(32 * 46, 32)
        self.dense1_add = nn.Linear(32 * 46 + len(col_features), 32)
        self.dense2 = nn.Linear(32, 32)

        self.dense_final = nn.Linear(32, num_classes)

    def forward(self, x, x_add=None):
        residual= self.conv(x)

        #block1
        x = F.relu(self.conv_pad(residual))
        x = self.conv_pad(x)
        x+= residual
        x = F.relu(x)
        residual = self.maxpool(x)

        #block2
        x=F.relu(self.conv_pad(residual))
        x=self.conv_pad(x)
        x+=residual
        x= F.relu(x)
        residual = self.maxpool(x)

        #block3
        x=F.relu(self.conv_pad(residual))
        x=self.conv_pad(x)
        x+=residual
        x= F.relu(x)
        residual = self.maxpool(x)


        #block4
        x=F.relu(self.conv_pad(residual))
        x=self.conv_pad(x)
        x+=residual
        x= F.relu(x)
        x= self.maxpool(x)

        #MLP
        x = x.view(-1, 32 * 46)
        if x_add is not None:
            x = torch.cat([x, x_add], axis=1)
            x = F.relu(self.dense1_add(x))
        else:
            x = F.relu(self.dense1(x))
        x= self.dense2(x)
        x = self.dense_final(x)
        return x

# train

In [ ]:
def run_one_fold(train_X, train_y, valid_X, valid_y, num_fold, train_X_add=None, valid_X_add = None):
    train_dataset = ECGDataset(train_X, train_X_add, train_y, transforms=get_train_transforms())
    valid_dataset = ECGDataset(valid_X, valid_X_add, valid_y, transforms=get_valid_transforms())
    train_loader = DataLoader(
        train_dataset,
        batch_size=DataLoaderConfig.batch_size,
        shuffle=True,
        num_workers=DataLoaderConfig.num_workers,
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=DataLoaderConfig.batch_size,
        shuffle=False,
        num_workers=DataLoaderConfig.num_workers,
    )

    fitter = Fitter(
        model=net,
        device=DEVICE,
        criterion=TrainConfig.criterion,
        n_epochs=TrainConfig.n_epochs,
        lr=TrainConfig.lr,
        sheduler=TrainConfig.scheduler,
        scheduler_params=TrainConfig.scheduler_params
    )
    fitter.fit(train_loader, valid_loader, num_fold=num_fold)

In [ ]:
class LossMeter:
    def __init__(self):
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

class AccMeter:
    def __init__(self):
        self.y_preds = []
        self.y_trues = []

    def update(self, y_true, y_pred):
        self.y_preds += list(y_pred.sigmoid().cpu().numpy().ravel())
        self.y_trues += list(y_true.cpu().numpy().ravel())

    def auc(self):
        if len(self.y_preds) == 0 or len(self.y_trues) == 0:
            return 0
        else:
            return roc_auc_score(self.y_trues, self.y_preds)

In [ ]:
class Fitter:
    def __init__(
        self, model, device, criterion, n_epochs,
        lr, sheduler=None, scheduler_params=None
    ):
        self.epoch = 0
        self.n_epochs = n_epochs
        self.base_dir = './'
        self.log_path = f'{self.base_dir}/log.txt'
        self.best_summary_loss = np.inf

        self.model = model
        self.device = device

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr, weight_decay=1e-5)

        if sheduler:
            self.scheduler = sheduler(self.optimizer, **scheduler_params)

        self.criterion = criterion().to(self.device)

        self.log(f'Fitter prepared. Device is {self.device}')

    def fit(self, train_loader, valid_loader, num_fold=0):
        for e in range(self.n_epochs):
            current_lr = self.optimizer.param_groups[0]['lr']
            self.log(f'\n{datetime.datetime.utcnow().isoformat()}\nLR: {current_lr}')

            t = int(time.time())
            summary_loss, final_scores = self.train_one_epoch(train_loader)
            self.log(
                f'[RESULT]: Train. Epoch: {self.epoch}, ' + \
                f'summary_loss: {summary_loss.avg:.5f}, ' + \
                f'final_score: {final_scores.auc():.5f}, ' + \
                f'time: {int(time.time()) - t} s'
            )

            t = int(time.time())
            summary_loss, final_scores = self.validation(valid_loader)
            self.log(
                f'[RESULT]: Valid. Epoch: {self.epoch}, ' + \
                f'summary_loss: {summary_loss.avg:.5f}, ' + \
                f'final_score: {final_scores.auc():.5f}, ' + \
                f'time: {int(time.time()) - t} s'
            )

            f_best = 0
            if summary_loss.avg < self.best_summary_loss:
                self.best_summary_loss = summary_loss.avg
                f_best = 1


            self.scheduler.step(metrics=summary_loss.avg)

            self.save(f'{self.base_dir}/last-checkpoint-{num_fold}.bin')

            if f_best:
                self.save(f'{self.base_dir}/best-checkpoint-{num_fold}.bin')
                print('New best checkpoint')

            self.epoch += 1

    def validation(self, val_loader):
        summary_loss = LossMeter()
        final_scores = AccMeter()

        t = int(time.time())
        for step, (images, add_feat, labels) in enumerate(val_loader):
            with torch.no_grad():
                labels = labels.unsqueeze(1).to(self.device)
                images = images.to(self.device)
                if add_feat is not None:
                    add_feat = add_feat.to(self.device)
                batch_size = images.shape[0]

                outputs = self.model(images, add_feat)
                loss = self.criterion(outputs, labels)

                final_scores.update(labels, outputs)
                summary_loss.update(loss.detach().item(), batch_size)
        return summary_loss, final_scores

    def train_one_epoch(self, train_loader):
        self.model.train()
        summary_loss = LossMeter()
        final_scores = AccMeter()

        t = int(time.time())
        for step, (images, add_feat, labels) in enumerate(train_loader):
            labels = labels.unsqueeze(1).to(self.device)
            images = images.to(self.device)
            if add_feat is not None:
                add_feat = add_feat.to(self.device)
            batch_size = images.shape[0]

            self.optimizer.zero_grad()
            outputs = self.model(images, add_feat)

            loss = self.criterion(outputs, labels)
            loss.backward()

            final_scores.update(labels.detach(), outputs.detach())
            summary_loss.update(loss.detach().item(), batch_size)

            self.optimizer.step()

        return summary_loss, final_scores

    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_summary_loss': self.best_summary_loss,
            'epoch': self.epoch,
        }, path)

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.best_summary_loss = checkpoint['best_summary_loss']
        self.epoch = checkpoint['epoch'] + 1

    def log(self, message):
        print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

def run_inference(test_X, num_fold, test_X_add=None):

    test_dataset = ECGDataset(
        X = test_X,
        X_add = test_X_add,
        transforms=get_valid_transforms(),
        train=False
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=DataLoaderConfig.batch_size,
        shuffle=False,
        num_workers=DataLoaderConfig.num_workers
    )

    checkpoint = torch.load(f'./best-checkpoint-{num_fold}.bin')
    net.load_state_dict(checkpoint['model_state_dict'])
    net.eval()
    print("model loaded")

    result = []
    for step, (images, add_feat) in enumerate(test_loader):
        print(step, end='\r')
        if add_feat is not None:
            add_feat = add_feat.to(DEVICE)
        y_pred = net(images.to(DEVICE), add_feat).detach().sigmoid().cpu().numpy().ravel()
        result.extend(y_pred)
    return np.array(result)

In [ ]:
class DataLoaderConfig:
    batch_size = 32
    num_workers = 8

class TrainConfig:
    criterion = nn.BCEWithLogitsLoss
    n_epochs = 15
    n_splits = 10
    lr = 0.001
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau
    # scheduler = torch.optim.lr_scheduler.StepLR
    scheduler_params = dict(
        mode='min',
        factor=0.5,
        patience=2,
        verbose=False,
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0,
        min_lr=1e-8,
        eps=1e-08
    )

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
seed_everything(SEED)

skf = StratifiedKFold(n_splits=TrainConfig.n_splits)
y_preds = np.zeros(len(df_train), np.float32)
y_trues = np.zeros(len(df_train), np.float32)
for fold, (train_index, valid_index) in enumerate(skf.split(np.arange(len(df_train)), y=df_train[col_target])):
    train_X, train_X_add, train_y, valid_X, valid_X_add, valid_y = ecg_train[train_index], df_train.iloc[train_index][col_features].values, target_train[train_index], ecg_train[valid_index], df_train.iloc[valid_index][col_features].values, target_train[valid_index]
    print('-'*30)
    print(f'fold: {fold}')
    net = Anomaly_Classifier(input_size=12, num_classes=1).to(DEVICE)
    run_one_fold(train_X, train_y, valid_X, valid_y, fold, train_X_add=train_X_add, valid_X_add=valid_X_add)

    y_pred = run_inference(valid_X, fold, test_X_add=valid_X_add)
    y_preds[valid_index] = y_pred
    y_trues[valid_index] = valid_y

cv = roc_auc_score(y_trues, y_preds)
print(f'AUC CV: {cv}')

# submit

In [ ]:
y_preds_test = []
for fold in range(TrainConfig.n_splits):
    y_pred = run_inference(ecg_test, fold, test_X_add=df_test[col_features].values)
    y_preds_test += [y_pred]
y_preds_test_mean = np.array(y_preds_test).mean(axis=0)

In [ ]:
df_sub = pd.read_csv(f'{data_dir}/sample_submission.csv')
df_sub['target'] = y_preds_test_mean
df_sub.to_csv("submission.csv", index=None)
df_sub.head()